<a href="https://colab.research.google.com/github/MemphisMeng/BigQuery-AI---Building-the-Future-of-Data/blob/main/Intelligent_Spam_Detective_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Necessary packages


In [1]:
# # Install all necessary libraries, including BigQuery client
# !pip install "bigframes==2.9.0"
# !pip install "google-cloud-bigquery[bqstorage,pandas]==3.33.0"
# !pip install "google-cloud-bigquery-storage==2.30.0"
# !pip install "pandas==2.2.2"
# !pip install "protobuf==5.29.1" "rich==13.9.2"

## datasets


---
The referenced dataset is publicly accessible on kaggle, credit ML team in UCI.


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# input datasets
import pandas as pd
import kagglehub

################################################################################################################
# ONLY RUN THIS CELL WHEN YOU RUN THIS NOTEBOOK FOR THE FIRST TIME IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.#
# kagglehub.login()                                                                                            #
################################################################################################################

dataset_path = 'uciml/sms-spam-collection-dataset'
dataset_directory = kagglehub.dataset_download(dataset_path)
sms_df = pd.read_csv(f"{dataset_directory}/spam.csv", encoding="latin1")

Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.


In [4]:
# data cleansing
## combine all content from the 2nd to 5th column
sms_df['message'] = None
sms_df['message'].loc[sms_df['Unnamed: 4'].notna()] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str) + ',' + sms_df['Unnamed: 4'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 4'].isna()) & (sms_df['Unnamed: 3'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str) + ',' + sms_df['Unnamed: 3'].astype(str)

sms_df['message'].loc[(sms_df['Unnamed: 3'].isna()) & (sms_df['Unnamed: 2'].notna())] = \
  sms_df['v2'] + ',' + sms_df['Unnamed: 2'].astype(str)

sms_df['message'].loc[sms_df['Unnamed: 2'].isna()] = sms_df['v2']

# rename the column to make it meaningful
sms_df.rename(columns={'v1': 'label'}, inplace=True)

# take the necessary columns
sms_df = sms_df[['message', 'label']]

# Modeling

---
The most instinctive approach with the help of BigQuery AI is to ask it whether the give message is a piece of spam or ham, by providing a structured prompt. To do this, a `GeminiTextGenerator` object will be initiated.

In [5]:
import bigframes
from bigframes.ml.llm import GeminiTextGenerator

PROJECT_ID = "chatbot-278522"  # ⚠️ CHANGE THIS to your actual Google Cloud project ID
bigframes.options.bigquery.project = PROJECT_ID
gemini = GeminiTextGenerator()

Each of the messages in the BigQuery DataFrame is going to be questioned: "`Is the message spam or ham? Please only answer \"spam\" or \"ham\" in ONE word.`"

In [6]:
gemini_sms_df = gemini.predict(
    sms_df,
    prompt=["Is the message spam or ham? Please only answer \"spam\" or \"ham\" in ONE word.",
            sms_df['message']])

In [7]:
accuracy = (gemini_sms_df['label'] == gemini_sms_df['ml_generate_text_llm_result'].str.strip()).sum() / len(gemini_sms_df)
print(f"Accuracy of Gemini model: {accuracy:.4f}")

Accuracy of Gemini model: 0.7541


Based on the accuracy performance, there is space for optimization. To more accurately fit the pattern(s) between the texts and the label of spam/ham, plain texts are embedded into multi-dimensional vectors by `bigframes.ml.llm.TextEmbeddingGenerator` to prepare my further in-house modeling.

In [8]:
from bigframes.ml.llm import TextEmbeddingGenerator
from bigframes.ml.preprocessing import LabelEncoder

## preprocessing
# message is "X"
embedder = TextEmbeddingGenerator()
embedded_X = embedder.predict(sms_df['message'])
print("Input texts are successfully vectorized!")
# label is "y"
transformer = LabelEncoder(max_categories=2)
transformed_y = transformer.fit_transform(sms_df['label'])
print("Outcome labels are successfully encoded!")

Input texts are successfully vectorized!


Outcome labels are successfully encoded!


Due to the non-linear relationship between the inputs (natural language texts) and outputs (labels), and its robustness against the overfitting, **XGBoost** is chosen as the underlying classification algorithm, which is encapsulated and provided by `bigframes.ml`.

In [11]:
from bigframes.ml.ensemble import XGBClassifier
from bigframes.ml.model_selection import cross_validate

KFold = 3

clf = XGBClassifier()
cv_results = cross_validate(clf, X=embedded_X['ml_generate_embedding_result'], y=transformed_y['labelencoded_label'], cv=KFold)['test_score']

In [15]:
avg_accuracy = sum([cv_result['accuracy'] for cv_result in cv_results]) / KFold
print(f"Average accuracy of self-trained XGBoost Classification: {avg_accuracy[0]:.4f}")


Average accuracy of self-trained XGBoost Classification: 0.9833


Now that a rise of accuracy is witnessed, it is safe to say that the trained model is reliable to be reused. So we can retrain the model on the full dataset and save it on further usage.

In [30]:
# re-train through the full dataset
clf.fit(
    X=embedded_X['ml_generate_embedding_result'],
    y=transformed_y['labelencoded_label']
)
clf.to_gbq(model_name='chatbot-278522.retail_Business_Intelligence.xgboost_spam_detector')
# alone with other models
embedder.to_gbq(model_name='chatbot-278522.retail_Business_Intelligence.embedder')
transformer.to_gbq(model_name='chatbot-278522.retail_Business_Intelligence.label_encoder')

LabelEncoder(max_categories=2, min_frequency=0)

There is BigQuery-native (re)usage of the pre-trained XGBoost model: I let `GeminiTextGenerator` to randomly generate some SMS texts, and saved them to a dataset hosted on BigQuery. This simulates a data source that needs to be verified if each message is spam or ham.

In [31]:
import bigframes.pandas as bpd

generated_df = gemini.predict(
    bpd.DataFrame({"dummy": range(5)}),  # 5 rows → 5 generations,
    prompt="""
    Generate one and ONLY one SMS text message someone might receive.
    It can be a piece of SPAM or HAM (legitimate text).
    It doesn't have to be an ice breaker, instead, something that usually only appear in the middle of a conversation works.
    Just output the messages for me.
    Don't put any other text before or after the message, for example, "Here are a few distinct examples:".
    Make sure the message is eligible to English speakers, not truncated and distinct.
    You may add some emotional elements to the tone, perhaps emojis would help.
    """
)
generated_df.rename(columns={'ml_generate_text_llm_result': 'generated_message'}, inplace=True)
generated_df[['generated_message']].to_gbq(
    destination_table='chatbot-278522.retail_Business_Intelligence.generated_sms',
    if_exists='replace')

'chatbot-278522.retail_Business_Intelligence.generated_sms'

The trained XGBoosted model can tell whether or not the generated texts are spam or ham. We can do this in a completely BigQuery-native manner.

In [52]:
from google.cloud import bigquery

DATASET_ID = "retail_Business_Intelligence"        # This will be created for you
client = bigquery.Client(project=PROJECT_ID)

query = f'''
  SELECT content as message,
  CASE predicted_labelencoded_label
  WHEN 1 THEN 'ham'
  WHEN 0 THEN 'spam'
  END AS label
  FROM
  ML.PREDICT(
    MODEL `{PROJECT_ID}.{DATASET_ID}.xgboost_spam_detector`,
    (
      SELECT ml_generate_embedding_result, content
      FROM ML.GENERATE_EMBEDDING( MODEL `{PROJECT_ID}.{DATASET_ID}.embedder`,
          (SELECT generated_message AS content FROM `{PROJECT_ID}`.`{DATASET_ID}`.`generated_sms`)
          )
    )
  ) gen
'''

try:
    result_df = client.query(query).to_dataframe()
except Exception as e:
    print(f"❌ Error accessing BigQuery: {e}")
result_df

,message,label
0,"Okay, I'm on my way, ETA 15 mins 🏃‍♀️ Traffic'...",ham
1,"Okay, sounds good! I'll give you one single SM...",ham
2,"Okay, got it.\n\nSounds good, I'll bring the c...",ham
3,"Okay, I'm on my way! 🏃‍♀️ ETA 15 mins. Traffic...",ham
4,"Okay, I'm on my way! 🏃‍♀️ See you in 10! 😊\n",ham


# So, is that the end?


---

Definitely no! With the detecting tech gets more and more sensitive, the scammers player harder and harder to get. One of their tricks is to put what used to be their plain text messages in an image so that the traditional detection will be circumvented. In this section, I am going to explore the image comprehending and identifying capacities of GCP, with which a multi-modal spam detector will be developed and implemented.

In [ ]:
dataset_path = 'tiharajayawickrama/spam-misleading-images-dataset'
dataset_directory = kagglehub.dataset_download(dataset_path)

In [ ]:
dataset_directory